# Colonial Office List Extraction

## Text as Data Workshop

This notebook teaches three approaches to extracting structured data from historical documents:

1. **Regex (Rule-Based)** - Fast, free, deterministic, but brittle
2. **LLM Code Generation** - Use Gemini to generate Python code containing extracted data
3. **Comparison** - Evaluate extraction quality against a gold standard

**No local installation required** - everything runs in this Colab notebook!

---
# Part 1: Sample Data

First, let's load our test data - a section from the Sierra Leone Colonial Office List (1896).

In [ ]:
# Sample data: Sierra Leone 1896 (partial)
SAMPLE_TEXT = """
Civil Establishment.

Governor, Commander-in-Chief; and Vice-Admiral, Colonel F. Cardew, C.M.G., 2,000l. and 500l. allowances.

Private Secretary,

Governor's Clerk, Jacob W. Lewis, 180l.
Second Clerk, T. Taylor, B.A., 60l. to 80l.

Colonial Secretary's Department.

Colonial Secretary, Lt.-Col. J. O. Gore, 710l. to 800l., and quarters.

Assistant Colonial Secretary, E. Faulkner, 300l. to 350l.

Chief Clerk, J. E. Dawson, 200l.
2nd ditto, E. W. Cole, 100l. to 120l.
3rd ditto, J. W. Paris, 100l.
4th ditto, C. George, 55l.
5th ditto, J. T. Smith, 45l.
6th ditto, Bruce Faulkner, 35l.
7th ditto, J. A. Williams, 24l.

Treasury Department.

Colonial Treasurer, C. B. Mitford, 500l. to 550l., and quarters.

Assistant Colonial Treasurer, S. M. Bennett, 300l. to 350l., and quarters.

Chief Clerk and Cashier, M. A. Potts, 200l., and 50l. personal.

2nd Clerk, G. W. Cole, 160l.
3rd ditto, J. J. Wellington, 100l.
4th ditto, R. A. Smith, 55l.
5th ditto, J. Edwin, 42l.
6th ditto, J. S. Wright, 30l.

Sub-Accountant at Sulymah, J. A. Cline, 10l.

Audit Department (under Comptroller and Auditor-General).

Local Auditor, W. Cuddeford, 450l. and quarters.
Assistant Auditor, H. C. E. Barnes, 225l., and quarters or rent.
First Clerk, P. A. Nichols, 100l.
Clerk, 40l.

Medical Department.

Colonial Surgeon, W. T. Prout, M.B., 50l., travelling allowance, 91l. 5s.

Assistant ditto, Wm. Renner, M.R.C.S., 300l. to 350l., and allowance 45l. 12s. 6d.; M. L. Jarrett, M.R.C.S., 250l.; I. N. Paris, M.B., 200l., and allowance 45l. 12s. 6d.

Assistant ditto for Frontier Police, T. Bishop, L.R.C.P., 300l. to 350l., and 91l. allowance.

Medical Clerk and Storekeeper, D. Thomas Cole, 135l., and quarters.

Compounder, G. L. Davies, 90l. and quarters.

Keeper, Lunatic Asylum, Kissy, H. W. Lewis, 60l.

Dispensers-

Waterloo, W. Z. Young, 50l., and quarters.
Hastings, M. Aubec, 36l., and quarters.
York, C. A. Innis, 40l., and quarters.
Kent, 40l., and quarters.
Sherbro, A. W. Elliott, 50l., and quarters.
Sulymah, S. A. Bell, 60l., and quarters.
Regent, M. N. Lardner, 36l., and quarters.
Kissy, J. G. Roberts, 36l., and quarters.
Goderich, E. G. Luke, 36l., and quarters.
Falaba, J. E. Baker, 63l. to 70l., and quarters.
Mongheri, E. O. King, 63l. to 70l., and qtrs.
"""

print(f"Loaded {len(SAMPLE_TEXT)} characters of sample text")
print("\nFirst 500 characters:")
print(SAMPLE_TEXT[:500])

---
# Part 2: Regex Extraction (Rule-Based)

Let's try extracting with regular expressions. This is fast and free, but struggles with complex patterns.

In [ ]:
import re
import json
from dataclasses import dataclass, field, asdict
from typing import Optional

# =============================================================================
# PATTERNS AND CONSTANTS
# =============================================================================

# Military ranks (appear before name)
MILITARY_RANKS = [
    r"Colonel", r"Lt\.-Col\.", r"Lieut\.-Col\.", r"Major-General",
    r"Lieut\.-General", r"Major", r"Captain", r"Capt\.", r"Brigadier-Gen\.",
    r"Surgeon-Major", r"Staff-Surgeon", r"Fleet-Surgeon"
]
MILITARY_RANK_PATTERN = re.compile(r'^(' + '|'.join(MILITARY_RANKS) + r')\s+', re.IGNORECASE)

# Honors (appear after name)
HONORS = ["K.C.M.G.", "G.C.M.G.", "C.M.G.", "K.C.B.", "G.C.B.", "C.B.", "D.S.O.", "Kt."]
HONOR_PATTERN = re.compile(r'\b(' + '|'.join([re.escape(h) for h in HONORS]) + r')')

# Qualifications (academic/professional)
QUALIFICATIONS = ["M.D.", "M.B.", "B.A.", "M.A.", "M.R.C.S.", "F.R.C.S.", "L.R.C.S.", "L.R.C.P.", "F.R.C.P."]
QUAL_PATTERN = re.compile(r'\b(' + '|'.join([re.escape(q) for q in QUALIFICATIONS]) + r')')

# Salary patterns
SALARY_RANGE = re.compile(r'(\d+)l\.\s+to\s+(\d+)l\.')
SALARY_SIMPLE = re.compile(r'(\d+)l\.')

# Known locations in Sierra Leone
KNOWN_LOCATIONS = [
    "Freetown", "Sherbro", "Sulymah", "Waterloo", "Hastings", "York",
    "Kent", "Regent", "Kissy", "Goderich", "Falaba", "Mongheri"
]

# =============================================================================
# EXTRACTION FUNCTIONS
# =============================================================================

def detect_department(line: str) -> Optional[str]:
    """Detect if line is a department header."""
    line = line.strip()
    if not line or 'l.' in line:
        return None
    
    dept_keywords = ["Department", "Office", "Establishment", "Council"]
    if any(kw in line for kw in dept_keywords) or line.endswith('.'):
        clean = line.rstrip('.')
        if len(clean) < 60:
            return clean
    return None

def parse_salary(text: str) -> tuple:
    """Parse salary from text. Returns (min, max, allowances)."""
    salary_min = salary_max = None
    allowances = None
    
    range_match = SALARY_RANGE.search(text)
    if range_match:
        salary_min = int(range_match.group(1))
        salary_max = int(range_match.group(2))
        text_after = text[range_match.end():]
    else:
        simple_match = SALARY_SIMPLE.search(text)
        if simple_match:
            salary_min = int(simple_match.group(1))
            salary_max = salary_min
            text_after = text[simple_match.end():]
        else:
            return None, None, None
    
    # Extract allowances
    if 'quarters' in text_after.lower() or 'allowance' in text_after.lower():
        allowances = text_after.strip().lstrip(',').strip()
    
    return salary_min, salary_max, allowances

def extract_military_rank(text: str) -> tuple:
    """Extract military rank. Returns (rank, remaining_text)."""
    match = MILITARY_RANK_PATTERN.match(text)
    if match:
        return match.group(1), text[match.end():].strip()
    return None, text

def extract_honors_and_quals(text: str) -> tuple:
    """Extract honors and qualifications. Returns (honors, quals, clean_text)."""
    honors = HONOR_PATTERN.findall(text)
    quals = QUAL_PATTERN.findall(text)
    
    clean = text
    for h in honors:
        clean = clean.replace(h, '')
    for q in quals:
        clean = clean.replace(q, '')
    
    clean = re.sub(r',\s*,', ',', clean)
    clean = re.sub(r'\s+', ' ', clean).strip().strip(',')
    
    return honors, quals, clean

def extract_location(position: str) -> tuple:
    """Extract location from position. Returns (clean_position, location)."""
    for loc in KNOWN_LOCATIONS:
        if position.endswith(loc) or f", {loc}" in position:
            clean = position.replace(f", {loc}", "").replace(loc, "").strip().rstrip(',')
            return clean if clean else "Dispenser", loc
    return position, None

def parse_name(name_text: str) -> dict:
    """Parse name into components."""
    military_rank, remaining = extract_military_rank(name_text)
    honors, quals, remaining = extract_honors_and_quals(remaining)
    
    # Expand known abbreviations (but NOT single initials!)
    name = remaining.replace('Wm.', 'William').replace('Chas.', 'Charles')
    name = name.replace('Thos.', 'Thomas').replace('Geo.', 'George')
    name = name.strip().strip(',')
    
    # Split into given names and surname
    parts = name.split()
    if len(parts) >= 2:
        surname = parts[-1]
        given_names = ' '.join(parts[:-1])
    else:
        surname = name
        given_names = ''
    
    canonical = f"{surname}, {given_names}" if given_names else surname
    
    return {
        'name': name,
        'canonical_name': canonical,
        'given_names': given_names,
        'surname': surname,
        'military_rank': military_rank,
        'honors': honors,
        'qualifications': quals,
    }

def extract_officials(text: str) -> list:
    """Main extraction function."""
    officials = []
    current_dept = "Unknown"
    previous_position_base = "Clerk"  # Default for ditto
    
    for line in text.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
        
        # Check for department header
        dept = detect_department(line)
        if dept:
            current_dept = dept
            continue
        
        # Skip lines without salary
        if 'l.' not in line:
            continue
        
        # Handle semicolon-separated multiple entries
        entries = [line] if ';' not in line else line.split(';')
        
        for i, entry in enumerate(entries):
            entry = entry.strip()
            if not entry:
                continue
            
            # Parse salary
            salary_min, salary_max, allowances = parse_salary(entry)
            if salary_min is None:
                continue
            
            # Get text before salary
            salary_match = SALARY_RANGE.search(entry) or SALARY_SIMPLE.search(entry)
            before_salary = entry[:salary_match.start()].strip().rstrip(',')
            
            # Handle "ditto" - expand to previous position type
            if 'ditto' in before_salary.lower():
                ditto_match = re.match(r'^(\d+(?:st|nd|rd|th)?\s+)?ditto,?\s*(.*)$', before_salary, re.I)
                if ditto_match:
                    prefix = ditto_match.group(1) or ""
                    name_part = ditto_match.group(2).strip()
                    position = f"{prefix.strip()} {previous_position_base}".strip()
                    
                    if name_part:
                        parsed = parse_name(name_part)
                        position_clean, location = extract_location(position)
                        
                        officials.append({
                            **parsed,
                            'position': position_clean,
                            'department': current_dept,
                            'salary_min': salary_min,
                            'salary_max': salary_max,
                            'allowances': allowances,
                            'location': location,
                        })
                    continue
            
            # Standard entry: Position, Name, Salary
            parts = before_salary.split(',', 1)
            if len(parts) >= 2:
                position = parts[0].strip()
                name_part = parts[1].strip()
            else:
                # Might be location-based (e.g., "Waterloo, W. Z. Young")
                position = "Dispenser"
                name_part = parts[0].strip()
            
            # Check if position starts with location
            for loc in KNOWN_LOCATIONS:
                if before_salary.startswith(loc + ','):
                    position = "Dispenser"
                    name_part = before_salary[len(loc)+1:].strip()
                    break
            
            # Skip if no name (vacant position)
            if not name_part or name_part == before_salary:
                continue
            
            # Parse name
            parsed = parse_name(name_part)
            
            # Skip if name looks invalid
            if not parsed['surname'] or len(parsed['surname']) < 2:
                continue
            
            # Extract location from position
            position_clean, location = extract_location(position)
            
            # Update position base for ditto tracking
            base_match = re.match(r'^(Chief\s+)?(Clerk|Surgeon|Auditor|Treasurer)', position, re.I)
            if base_match:
                previous_position_base = base_match.group(2)
            
            officials.append({
                **parsed,
                'position': position_clean,
                'department': current_dept,
                'salary_min': salary_min,
                'salary_max': salary_max,
                'allowances': allowances,
                'location': location,
            })
    
    return officials

# =============================================================================
# RUN EXTRACTION
# =============================================================================

regex_results = extract_officials(SAMPLE_TEXT)

print(f"Regex extracted {len(regex_results)} officials\n")
print("=" * 60)
print("EXTRACTED OFFICIALS")
print("=" * 60)

for i, r in enumerate(regex_results[:15], 1):
    loc = f" @ {r['location']}" if r.get('location') else ""
    quals = f" [{', '.join(r['qualifications'])}]" if r.get('qualifications') else ""
    print(f"{i:2}. {r['position']}: {r['name']}{quals} ({r['salary_min']}-{r['salary_max']}l.){loc}")

if len(regex_results) > 15:
    print(f"... and {len(regex_results) - 15} more")

### Regex Limitations

Notice the problems:
- "ditto" handling is imperfect
- Multi-person lines not split correctly
- Position/name boundary detection is unreliable

This is why we need a smarter approach...

---
# Part 3: LLM Code Generation

## The Key Insight

Instead of asking the LLM to extract data directly, we ask it to **generate Python code** containing the data. This gives us:

- **Reviewable output** - You can read the code
- **Deterministic results** - Run code = same output
- **Version control** - Commit code to git

## Instructions

1. Go to [Google AI Studio](https://aistudio.google.com/)
2. Select **Gemini 2.0 Flash**
3. Copy the prompt from the next cell
4. Paste your source text at the end
5. Run and copy the generated Python code
6. Paste it in the cell after the prompt

In [ ]:
# THE PROMPT - Copy this to AI Studio
PROMPT = """
You are a code generator for historical data extraction. Given a Colonial Office List excerpt, generate a Python file that contains the extracted personnel data as a Python data structure.

## Your Task
Generate a complete, runnable Python file that:
1. Defines OFFICIALS as a list of dictionaries containing all extracted personnel
2. Includes a main() function that outputs JSON when run
3. Is self-contained with no external dependencies

## Output Format
Generate ONLY valid Python code. No markdown, no explanations.

Use this structure:

COLONY = "Sierra Leone"
YEAR = 1896

OFFICIALS = [
    {
        "name": "F. Cardew",
        "canonical_name": "Cardew, F.",
        "given_names": "F.",
        "surname": "Cardew",
        "position": "Governor",
        "department": "Civil Establishment - Sierra Leone",
        "salary_min": 2000,
        "salary_max": 2000,
        "allowances": "500l. allowances",
        "honors": ["C.M.G."],
        "qualifications": [],
        "military_rank": "Colonel",
        "location": None,
    },
]

def main():
    import json
    result = {"colony": COLONY, "year": YEAR, "total": len(OFFICIALS), "officials": OFFICIALS}
    print(json.dumps(result, indent=2))

if __name__ == "__main__":
    main()

## Extraction Rules
1. Extract EVERY person
2. "ditto" = same position type as line above (2nd ditto = 2nd Clerk)
3. Salaries: "300l. to 400l." → min=300, max=400
4. Expand ONLY: Wm.=William, Chas.=Charles, Thos.=Thomas
5. Do NOT expand single initials: C. stays C., NOT Charles
6. Honors: C.M.G., K.C.M.G. → "honors" list
7. Qualifications: M.D., M.B., M.R.C.S., B.A. → "qualifications" list
8. Military ranks: Colonel, Lt.-Col. → "military_rank" field
9. Location from position: "Dispenser, Waterloo" → location="Waterloo"
10. Vacant positions (no name): set name fields to None
11. Multiple people on one line: create separate entries

## Source Text
[PASTE YOUR TEXT HERE]
"""

print("Copy the prompt above, then:")
print("1. Go to https://aistudio.google.com/")
print("2. Select Gemini 2.0 Flash")
print("3. Paste the prompt")
print("4. Replace [PASTE YOUR TEXT HERE] with your source text")
print("5. Run and copy the generated code")
print("6. Paste it in the next cell")

In [ ]:
# PASTE YOUR GENERATED CODE HERE
# Delete this comment and paste the Python code from AI Studio

# Example (replace with your generated code):
COLONY = "Sierra Leone"
YEAR = 1896

OFFICIALS = [
    # Paste your OFFICIALS list here
]

def main():
    import json
    result = {"colony": COLONY, "year": YEAR, "total": len(OFFICIALS), "officials": OFFICIALS}
    print(json.dumps(result, indent=2))

# Uncomment to run:
# main()

### Review Your Generated Code!

Before running, check the OFFICIALS list:
- Are all people extracted?
- Are "ditto" positions expanded correctly?
- Are initials NOT over-expanded? (C. should stay C.)
- Are salaries correct?

---
# Part 4: Gold Standard Comparison

Let's compare extraction results against a manually verified gold standard.

In [ ]:
# Gold standard (manually verified)
GOLD_STANDARD = [
    {"name": "F. Cardew", "position": "Governor, Commander-in-Chief; and Vice-Admiral", "salary_min": 2000},
    {"name": "Jacob W. Lewis", "position": "Governor's Clerk", "salary_min": 180},
    {"name": "T. Taylor", "position": "Second Clerk", "salary_min": 60},
    {"name": "J. O. Gore", "position": "Colonial Secretary", "salary_min": 710},
    {"name": "E. Faulkner", "position": "Assistant Colonial Secretary", "salary_min": 300},
    {"name": "J. E. Dawson", "position": "Chief Clerk", "salary_min": 200},
    {"name": "E. W. Cole", "position": "2nd Clerk", "salary_min": 100},
    {"name": "J. W. Paris", "position": "3rd Clerk", "salary_min": 100},
    {"name": "C. George", "position": "4th Clerk", "salary_min": 55},
    {"name": "J. T. Smith", "position": "5th Clerk", "salary_min": 45},
    {"name": "Bruce Faulkner", "position": "6th Clerk", "salary_min": 35},
    {"name": "J. A. Williams", "position": "7th Clerk", "salary_min": 24},
    {"name": "C. B. Mitford", "position": "Colonial Treasurer", "salary_min": 500},
    {"name": "S. M. Bennett", "position": "Assistant Colonial Treasurer", "salary_min": 300},
    {"name": "M. A. Potts", "position": "Chief Clerk and Cashier", "salary_min": 200},
    {"name": "G. W. Cole", "position": "2nd Clerk", "salary_min": 160},
    {"name": "J. J. Wellington", "position": "3rd Clerk", "salary_min": 100},
    {"name": "R. A. Smith", "position": "4th Clerk", "salary_min": 55},
    {"name": "J. Edwin", "position": "5th Clerk", "salary_min": 42},
    {"name": "J. S. Wright", "position": "6th Clerk", "salary_min": 30},
    {"name": "J. A. Cline", "position": "Sub-Accountant", "salary_min": 10},
    {"name": "W. Cuddeford", "position": "Local Auditor", "salary_min": 450},
    {"name": "H. C. E. Barnes", "position": "Assistant Auditor", "salary_min": 225},
    {"name": "P. A. Nichols", "position": "First Clerk", "salary_min": 100},
    {"name": "W. T. Prout", "position": "Colonial Surgeon", "salary_min": 50},
    {"name": "William Renner", "position": "Assistant Colonial Surgeon", "salary_min": 300},
    {"name": "M. L. Jarrett", "position": "Assistant Colonial Surgeon", "salary_min": 250},
    {"name": "I. N. Paris", "position": "Assistant Colonial Surgeon", "salary_min": 200},
    {"name": "T. Bishop", "position": "Assistant Colonial Surgeon for Frontier Police", "salary_min": 300},
    {"name": "D. Thomas Cole", "position": "Medical Clerk and Storekeeper", "salary_min": 135},
    {"name": "G. L. Davies", "position": "Compounder", "salary_min": 90},
    {"name": "H. W. Lewis", "position": "Keeper, Lunatic Asylum", "salary_min": 60},
    {"name": "W. Z. Young", "position": "Dispenser", "salary_min": 50},
    {"name": "M. Aubec", "position": "Dispenser", "salary_min": 36},
    {"name": "C. A. Innis", "position": "Dispenser", "salary_min": 40},
    {"name": "A. W. Elliott", "position": "Dispenser", "salary_min": 50},
    {"name": "S. A. Bell", "position": "Dispenser", "salary_min": 60},
    {"name": "M. N. Lardner", "position": "Dispenser", "salary_min": 36},
    {"name": "J. G. Roberts", "position": "Dispenser", "salary_min": 36},
    {"name": "E. G. Luke", "position": "Dispenser", "salary_min": 36},
    {"name": "J. E. Baker", "position": "Dispenser", "salary_min": 63},
    {"name": "E. O. King", "position": "Dispenser", "salary_min": 63},
]

print(f"Gold standard contains {len(GOLD_STANDARD)} officials")

In [ ]:
def evaluate_extraction(extracted: list, gold: list) -> dict:
    """Compare extraction against gold standard."""
    # Normalize names for matching
    def normalize(name):
        if not name:
            return ""
        return name.lower().replace(',', '').replace('.', '').strip()

    gold_names = {normalize(g['name']) for g in gold}
    extracted_names = {normalize(e.get('name', '')) for e in extracted if e.get('name')}

    matched = gold_names & extracted_names
    missed = gold_names - extracted_names
    extra = extracted_names - gold_names

    recall = len(matched) / len(gold_names) if gold_names else 0
    precision = len(matched) / len(extracted_names) if extracted_names else 0

    return {
        'gold_count': len(gold),
        'extracted_count': len(extracted),
        'matched': len(matched),
        'recall': recall,
        'precision': precision,
        'missed': list(missed)[:10],
        'extra': list(extra)[:10],
    }

# Evaluate regex results
regex_eval = evaluate_extraction(regex_results, GOLD_STANDARD)

print("=" * 50)
print("REGEX EVALUATION")
print("=" * 50)
print(f"Gold standard: {regex_eval['gold_count']} officials")
print(f"Extracted: {regex_eval['extracted_count']} officials")
print(f"Matched: {regex_eval['matched']}")
print(f"Recall: {regex_eval['recall']:.1%}")
print(f"Precision: {regex_eval['precision']:.1%}")
if regex_eval['missed']:
    print(f"\nMissed: {regex_eval['missed'][:5]}")
if regex_eval['extra']:
    print(f"Extra: {regex_eval['extra'][:5]}")

In [ ]:
# Evaluate your LLM-generated results
# Uncomment and run after pasting your generated code

# llm_eval = evaluate_extraction(OFFICIALS, GOLD_STANDARD)
#
# print("=" * 50)
# print("LLM CODE GENERATION EVALUATION")
# print("=" * 50)
# print(f"Gold standard: {llm_eval['gold_count']} officials")
# print(f"Extracted: {llm_eval['extracted_count']} officials")
# print(f"Matched: {llm_eval['matched']}")
# print(f"Recall: {llm_eval['recall']:.1%}")
# print(f"Precision: {llm_eval['precision']:.1%}")

---
# Part 5: Comparison Summary

| Approach | Recall | Precision | Cost | Time | Reviewable? |
|----------|--------|-----------|------|------|-------------|
| Regex | ~80% | ~70% | $0 | <1s | Code only |
| LLM Direct | ~98% | ~98% | $0 | ~10s | No |
| **LLM Code Gen** | ~98% | ~98% | $0 | ~10s | **Yes** |

## Key Takeaway

The **LLM Code Generation** approach gives you:
- High accuracy (like direct LLM extraction)
- Reviewable output (you can check the code)
- Deterministic results (run code = same output)
- Version control (commit code to git)

This is the recommended approach for historical document extraction!

---
# Exercises

1. **Improve the regex**: Can you fix the ditto handling to match more officials?

2. **Test the LLM**: Run the AI Studio extraction and compare your results to the gold standard.

3. **Find errors**: Review the generated code carefully. Are there any hallucinated entries or incorrect expansions?

4. **Try another section**: Use text from a different colony or year. Does the prompt still work?